# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

In [1]:
import os

db_path = "data"

# Listar subdirectorios y archivos relevantes
for root, dirs, files in os.walk(db_path):
    print(f"Directorio: {root}")
    for file in files:
        if file.endswith(".lance"):
            print(f"  Tabla encontrada: {file}")


Directorio: data
Directorio: data/ANN
Directorio: data/ANN/mis_vectores.lance
Directorio: data/ANN/mis_vectores.lance/_transactions copy
Directorio: data/ANN/mis_vectores.lance/_versions
Directorio: data/ANN/mis_vectores.lance/data copy
  Tabla encontrada: 44eacc30-f76d-4aee-9547-0064dcdf8cf5.lance
Directorio: data/ANN/mis_vectores.lance/_indices copy
Directorio: data/ANN/mis_vectores.lance/_indices copy/6defcf63-6466-4051-8cfa-6dabaf5e196e
Directorio: data/ANN/mis_vectores.lance/_indices copy/a3ce7190-1891-4c48-bc91-e5814c4ae795
Directorio: data/ANN/mis_vectores.lance/_indices copy/7eab73c5-a744-4ad2-b4d8-06effda5b747
Directorio: data/ANN/mis_vectores.lance/_versions copy
Directorio: data/ANN/mis_vectores.lance/_transactions
Directorio: data/ANN/mis_vectores.lance/data
  Tabla encontrada: 6802c199-e43f-4a06-8cc9-6d4544734cbc.lance
Directorio: data/introdb
Directorio: data/introdb/multimodal_embeddings.lance
Directorio: data/introdb/multimodal_embeddings.lance/_versions
Directorio: dat

In [2]:
import os

# Obtener ruta absoluta de la tabla
absolute_path = os.path.abspath("data/ANN/mis_vectores.lance")
print("Ruta absoluta de la tabla:", absolute_path)


Ruta absoluta de la tabla: /Users/emiliahernandez/Desktop/LanceDB/Tareas/208115/data/ANN/mis_vectores.lance


Busqué mis vectores pero no me debaja trabajar con ella. La copie y todo y no jalo, entonces la volví a generar

In [5]:
import lancedb
import pandas as pd
import pyarrow as pa
import polars as pl
import numpy as np
import random
from lancedb.pydantic import Vector, LanceModel
from pydantic import BaseModel
# Configurar LanceDB
uri = "data/ANN2"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear 10,000 vectores de muestra aleatorios
np.random.seed(42)  # Para reproducibilidad
data = [
    {"vector": row, "item": f"item {i}"}
    for i, row in enumerate(np.random.random((19_999_0, 1536)).astype("float32"))
]

# Crear tabla en la base de datos
tbl = db.create_table("mis_vectores", data=data)
print(f"Tabla 'mis_vectores' creada con {len(data)} vectores.")
# Mostrar las primeras 5 filas de la tabla

Tabla 'mis_vectores' creada con 199990 vectores.


In [6]:
import numpy as np
import lancedb

# Conexión a la base de datos
uri = "data/ANN2"  # Directorio donde se guardó la base de datos
db = lancedb.connect(uri)

# Abrir la tabla `mis_vectores`
table = db.open_table("mis_vectores")
print("Tabla 'mis_vectores' cargada exitosamente.")

# **Paso 1**: Generar un vector aleatorio con la misma dimensión que los embeddings
# Extraer un vector de ejemplo para determinar la dimensión
sample_vector = table.to_pandas().iloc[0]["vector"]  # Acceder al primer embedding
vector_dim = len(sample_vector)

# Crear un vector aleatorio con la misma dimensión
random_vector = np.random.rand(vector_dim)
random_vector = random_vector / np.linalg.norm(random_vector)  # Normalizar

# **Paso 2**: Buscar los 5 elementos más cercanos
results = table.search(random_vector).limit(5).to_pandas()

# **Paso 3**: Proyectar resultados para mostrar solo las columnas `item` y `_distance`
projected_results = results[["item", "_distance"]]

# **Paso 4**: Excluir los elementos cuyo nombre sea "item 500"
filtered_results = projected_results[projected_results["item"] != "item 500"]

# Mostrar resultados finales
print("Resultados filtrados:")
print(filtered_results)


Tabla 'mis_vectores' cargada exitosamente.
Resultados filtrados:
          item   _distance
0    item 2319  427.060303
1   item 59917  431.943817
2  item 100836  433.208313
3  item 120194  434.301575
4  item 123504  435.128387


**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [8]:
import numpy as np
import lancedb
import pandas as pd

# Conexión a la base de datos LanceDB
db = lancedb.connect("data/introdb2")  # Cambia esta ruta a la de tu base de datos

# **Paso 1**: Crear un DataFrame con datos iniciales
# Inicializa con vectores vacíos como [0, 0, 0, 0]
initial_data = pd.DataFrame({
    "vector": [[0, 0, 0, 0]],  # Inicializar con un vector válido
    "nombre": ["placeholder"],  # Nombre genérico
    "categoria": ["placeholder"]  # Categoría genérica
})

# **Paso 2**: Crear la tabla vacía con un registro inicial
table = db.create_table("nueva_tabla", initial_data)
print("Tabla 'nueva_tabla' creada exitosamente")

# **Paso 3**: Insertar 5 registros en la tabla
data_to_insert = pd.DataFrame({
    "vector": [
        np.random.rand(4).tolist() for _ in range(5)  # Vectores aleatorios de 4 dimensiones
    ],
    "nombre": [f"nombre_{i}" for i in range(1, 6)],  # Nombres personalizados
    "categoria": [f"categoria_{i}" for i in range(1, 6)]  # Categorías personalizadas
})
table.add(data_to_insert)
print("Datos insertados exitosamente")

# **Paso 4**: Mostrar el contenido de la tabla
result = table.to_pandas()
print(result)


Tabla 'nueva_tabla' creada exitosamente
Datos insertados exitosamente
                                             vector       nombre    categoria
0                              [0.0, 0.0, 0.0, 0.0]  placeholder  placeholder
1    [0.86513436, 0.53423107, 0.6786794, 0.9053158]     nombre_1  categoria_1
2    [0.9805097, 0.58602875, 0.9148938, 0.13578984]     nombre_2  categoria_2
3   [0.4048784, 0.07962073, 0.68613964, 0.62683177]     nombre_3  categoria_3
4       [0.980636, 0.519533, 0.7820983, 0.14700311]     nombre_4  categoria_4
5  [0.46559915, 0.18441388, 0.46827078, 0.39827234]     nombre_5  categoria_5


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [9]:
import numpy as np
import lancedb
import pandas as pd

# **Paso 1**: Conexión a la base de datos LanceDB
db = lancedb.connect("data/task3")  # Cambia esta ruta a la de tu base de datos

# Crear un DataFrame inicial con las columnas especificadas
data = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],  # IDs
    "vector": [  # Vectores de 3 números flotantes
        [1.0, 2.0, 10.0],
        [4.0, 5.0, 6.0],
        [7.0, 8.0, 9.0],
        [0.0, 1.0, 2.0],
        [12.0, 4.0, 5.0]
    ]
})

# Crear la tabla en la base de datos
table = db.create_table("task3_tabla", data, mode="overwrite")
print("Tabla 'task3_tabla' creada exitosamente")

# **Paso 2**: Actualizar el vector de la fila donde id=3
# Extraer datos actuales como DataFrame
current_data = table.to_pandas()

# Actualizar el vector de la fila con id=3
# Usamos `.at` para acceder directamente a la celda
index_to_update = current_data[current_data["id"] == 3].index[0]
current_data.at[index_to_update, "vector"] = [10.0, 11.0, 10.0]

# Sobrescribir la tabla con los datos actualizados
table = db.create_table("task3_tabla", current_data, mode="overwrite")
print("Vector actualizado exitosamente")

# **Paso 3**: Filtrar filas donde al menos un valor del vector sea mayor a 9.0
filtered_data = current_data[
    current_data["vector"].apply(lambda vec: any(x > 9.0 for x in vec))
]

# Mostrar el resultado
print("Filas donde al menos un valor del vector es mayor a 9.0:")
print(filtered_data)


Tabla 'task3_tabla' creada exitosamente
Vector actualizado exitosamente
Filas donde al menos un valor del vector es mayor a 9.0:
   id              vector
0   1    [1.0, 2.0, 10.0]
2   3  [10.0, 11.0, 10.0]
4   5    [12.0, 4.0, 5.0]


[2024-12-03T14:50:14Z WARN  lance::dataset] No existing dataset at /Users/emiliahernandez/Desktop/LanceDB/Tareas/208115/data/task3/task3_tabla.lance, it will be created


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
from PIL import Image
from sentence_transformers import SentenceTransformer
import lancedb

# **Paso 1: Configuración del modelo**
# Modelo para texto
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Modelo para imágenes
image_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
image_model.fc = torch.nn.Identity()  # Retirar la capa final
image_model.eval()

# Transformaciones para imágenes
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# **Paso 2: Ruta a la carpeta de imágenes**
image_dir = "data/data/images"  # Ajusta la ruta según tu estructura de proyecto

# **Paso 3: Procesar cada imagen y generar embeddings**
image_embeddings = []
image_data = []

for image_file in os.listdir(image_dir):
    if image_file.endswith(".jpg") or image_file.endswith(".png"):
        img_path = os.path.join(image_dir, image_file)
        img = Image.open(img_path).convert("RGB")
        img_tensor = image_transform(img).unsqueeze(0)

        # Generar embedding de la imagen
        with torch.no_grad():
            embedding = image_model(img_tensor).squeeze().numpy().astype('float32')

        image_data.append(image_file)  # Nombre de la imagen
        image_embeddings.append(embedding.tolist())  # Embedding como lista

# **Paso 4: Generar embeddings para textos**
texts = [
    "La tecnología avanza rápido",
    "El cielo es azul y hermoso",
    "Los avances científicos son notables",
    "Una escena nocturna tranquila",
    "Un paisaje marino con tonos azules"
]

text_embeddings = [text_model.encode(text).tolist() for text in texts]

# **Paso 5: Crear DataFrame con texto, imágenes y embeddings**
df_multimodal = pd.DataFrame({
    "texto": texts,
    "imagen": image_data[:len(texts)],  # Emparejar imágenes con textos disponibles
    "embedding_texto": text_embeddings,
    "embedding_imagen": image_embeddings[:len(texts)]  # Asegurarse de que coincidan las longitudes
})

print("Embeddings multimodales generados:")
print(df_multimodal.head())

# **Paso 6: Guardar en LanceDB**
# Conexión a LanceDB
db = lancedb.connect("data/introdb")

# Crear tabla en LanceDB
tabla_multimodal = db.create_table("multimodal_embeddings", data=df_multimodal, mode="overwrite")
print("Tabla 'multimodal_embeddings' creada con éxito.")

# **Paso 7: Realizar búsqueda combinada**
# Embedding del texto objetivo
query_text = "La tecnología avanza rápido"
query_text_embedding = text_model.encode(query_text)

# Crear un embedding de 512 dimensiones para el color azul
query_image_embedding = np.zeros(512)
query_image_embedding[-1] = 1.0  # Predomina el azul

# Realizar búsqueda en la tabla
results = tabla_multimodal.to_pandas()

# Calcular distancia promedio entre embeddings
def calculate_combined_distance(row):
    text_distance = np.linalg.norm(np.array(row["embedding_texto"]) - np.array(query_text_embedding))
    image_distance = np.linalg.norm(np.array(row["embedding_imagen"]) - query_image_embedding)
    return (text_distance + image_distance) / 2

results["distance"] = results.apply(calculate_combined_distance, axis=1)

# Ordenar por la menor distancia promedio
results = results.sort_values("distance").reset_index(drop=True)

# **Paso 8: Mostrar los resultados combinados**
print("Resultados ordenados por menor distancia promedio:")
print(results[["texto", "imagen", "distance"]])


Embeddings multimodales generados:
                                  texto        imagen  \
0           La tecnología avanza rápido  imagen_3.jpg   
1            El cielo es azul y hermoso  imagen_2.jpg   
2  Los avances científicos son notables  imagen_1.jpg   
3         Una escena nocturna tranquila  imagen_5.jpg   
4    Un paisaje marino con tonos azules  imagen_4.jpg   

                                     embedding_texto  \
0  [0.013738801702857018, -0.015867607668042183, ...   
1  [0.0029471139423549175, 0.07605057209730148, -...   
2  [0.023409849032759666, -0.0009007313055917621,...   
3  [0.05347875878214836, 0.043729912489652634, -0...   
4  [-0.02923412248492241, 0.11961355060338974, -0...   

                                    embedding_imagen  
0  [0.5015254020690918, 0.09100812673568726, 0.88...  
1  [1.1300411224365234, 0.29969215393066406, 0.10...  
2  [0.03967608883976936, 1.3665385246276855, 0.12...  
3  [1.4056179523468018, 2.2068862915039062, 1.595...  
4  [1.2344

A pesar de que el renglon con id 2 tiene el texto igual al de la búsqueda, la imagen no tiene nada de azul, entonces no me lo marcó con una menor distancia